# Demo: Creating Content from Webpage

**Objective:**

This notebook demoes how a simple workflow of sequential requests to large language models can be leveraged to create multiple pieces of content from an arbitrary webpage.

This workflow consists of three sequential steps:
1. Generate a summary of desired webpage
2. Structure the summary according to a user-defined outline.
3. Create a social media post from structured summary with custom persona.

**Requirements:**

This demo uses the OpenAI Chat API and, thus, requires an OpenAI API key.

**Disclaimer:**

This notebook is strictly for educational purposes. Users should employ it responsibly, ensuring accuracy, respecting privacy and copyright, and avoiding the dissemination of misinformation. Always consider the ethical implications and context of the generated content.

## Setup: Load and parse HTML webpage

Before we can start our workflow, we extract the text content from the desired webpage, focusing on paragraphs and headers, to use downstream. We use the Llama 2 announcement from Meta AI in this demo.

In [22]:
# Extract text from webpage

import os
from IPython.display import Markdown
from genaipy.extractors.web import extract_text

os.chdir('..') # Change path to root directory

URL = "https://ai.meta.com/blog/large-language-model-llama-meta-ai/"
TAGS = ["p", "h1"]

text = extract_text(url=URL, tags=TAGS)
Markdown(text[:1200]) # Display first 1200 characters

Introducing LLaMA  A foundational  65 billion parameter large language model

February 24  2023

UPDATE  We just launched Llama 2   for more information on the latest see our blog post on Llama 2  

As part of Meta s commitment to open science  today we are publicly releasing LLaMA  Large Language Model Meta AI   a state of the art foundational large language model designed to help researchers advance their work in this subfield of AI  Smaller  more performant models such as LLaMA enable others in the research community who don t have access to large amounts of infrastructure to study these models  further democratizing access in this important  fast changing field 

Training smaller foundation models like LLaMA is desirable in the large language model space because it requires far less computing power and resources to test new approaches  validate others  work  and explore new use cases  Foundation models train on a large set of unlabeled data  which makes them ideal for fine tuning for a variety of tasks  We are making LLaMA available at several sizes  7B  13B  33B  and 65B parameters  and also sharing a LLaMA model card that details how we built the model in keeping with our app

## Step [1/3]: Generate Summary

In the first step of the workflow, we provide the input text sample to `GPT-3.5-turbo` and request it to generate a 150 words long summary. This step serves as the foundation for the following steps.

In [9]:

from genaipy.openai_apis.chat import get_chat_response
from genaipy.prompts.generate_summaries import build_summary_prompt

prompt = build_summary_prompt(text=text, max_words=150)

summary = get_chat_response(prompt=prompt, temperature=0.2)
Markdown(summary)

Meta AI has released LLaMA, a large language model designed to assist researchers in the field of AI. The model is smaller and more efficient, making it accessible to researchers who lack extensive resources. LLaMA is available in different sizes and can be fine-tuned for various tasks. Large language models have demonstrated their potential in generating creative text, solving mathematical problems, and answering comprehension questions. However, limited access to these models hinders research progress in understanding their workings and addressing issues like bias and misinformation. LLaMA aims to be versatile and applicable to different use cases. The model is released under a noncommercial license for research purposes, and access is granted on a case-by-case basis. Meta AI encourages collaboration in developing responsible guidelines for large language models.

## Step [2/3]: Structure Summary with Outline

In the second step of the workflow, we use the generated summary and utilize `GPT-3.5-turbo` to structure it according to a custom outline. Notably, this structured outline is completely customizable to the requirements of the use case (e.g., meeting minutes, reports, etc.).

Here, we opted for a basic outline for structuring summaries displayed below. Since the structured summary is in `Markdown`, it can be directly saved and displayed in many text editors.

In [15]:
# Load structured outline

from genaipy.outlines.basic_summary_outline import BASIC_SUMMARY_OUTLINE

tpl = BASIC_SUMMARY_OUTLINE.strip()
print(tpl)

# [Short Title]

## Summary
(Exactly 1 sentence long summary of the main topic and objective.)

## Key points 
(bulleted list of 3-5 main points.)
- [Key insight 1]
- [Key insight 2]
- [Key insight 3]
- ... (only if needed)

## Key concepts
(bulleted list of exactly 3-5 main concepts and entities.)
- [Key concept/entity 1]
- [Key concept/entity 2]
- [Key concept/entity 3]
- ... (only if needed)


In [16]:
# Generate structured summary

from genaipy.prompts.style_texts import build_style_prompt

style_prompt = build_style_prompt(text=summary, tpl=tpl)

styled_summary = get_chat_response(prompt=style_prompt, temperature=0) 
Markdown(styled_summary)

```
# Meta AI's LLaMA: A Versatile Language Model for AI Research

## Summary
Meta AI has developed LLaMA, a smaller and more efficient language model, to assist researchers in the field of AI by providing access to large language models for various tasks, while addressing issues like bias and misinformation.

## Key points
- LLaMA is a large language model designed to assist researchers in AI with limited resources.
- The model is smaller and more efficient, making it accessible to a wider range of researchers.
- LLaMA can be fine-tuned for different tasks and is available in various sizes.
- Large language models have shown potential in generating creative text, solving mathematical problems, and answering comprehension questions.
- Limited access to these models hampers research progress in understanding their workings and addressing issues like bias and misinformation.

## Key concepts
- LLaMA: A language model developed by Meta AI to assist researchers in AI.
- Fine-tuning: The process of adapting a pre-trained language model for specific tasks.
- Bias and misinformation: Issues that need to be addressed in large language models.
```

## Step [3/3]: Generate Social Media Post

In the final step of the workflow, we generate a social media post from the structured summary using `GPT-4-turbo`. 
Notably, we set a custom role for the LLM via the system message to tailor the style and tone of the post.

In this case, we set the LLM's role to a  professional content creator on Linkedin excelling at communicating complex AI topics in a simple way. Feel free to change the system message below, to explore different types of social media posts!

In [12]:
# Define custom role

system_message = "You are an expert content creator on LinkedIn. You excell at communicating complex topics in Generative AI in a simple and professional way."
Markdown(system_message)

You are an expert content creator on LinkedIn. You excell at communicating complex topics in Generative AI in a simple and professional way.

In [13]:
# Generate LinkedIn post

from genaipy.prompts.generate_posts import build_post_prompt

generate_prompt = build_post_prompt(text=styled_summary, max_words=150)

post = get_chat_response(
    prompt=generate_prompt,
    sys_message=system_message,
    temperature=0.5,
    model="gpt-4-1106-preview"
    )

Markdown(post)

🚀 Exciting News from Meta AI: Introducing LLaMA 🌟

As an AI enthusiast, I'm thrilled to share that Meta AI has launched a game-changer: LLaMA, a language model that's not just powerful but also efficient and accessible!

🔍 **Why LLaMA Stands Out:**
- Tailored for researchers with limited resources, LLaMA is here to democratize AI research.
- It's a versatile tool, ready to be fine-tuned for a spectrum of tasks.
- Expect LLaMA in various sizes, catering to different research needs.

💡 **Impact on AI Research:**
- LLaMA's efficiency could revolutionize creative text generation, complex problem-solving, and comprehension tasks.
- It's a step towards overcoming research barriers in understanding and improving language models, especially in tackling bias and misinformation.

I'm eager to see how LLaMA will shape the future of AI research. Let's discuss how this innovation can unlock new possibilities! #AILaMA #MetaAI #LanguageModels #AIResearch